# google drive

In [1]:
from google.colab import drive 
import os
drive.mount("/content/gdrive", force_remount=True)
os.chdir("/content/gdrive/MyDrive/xai-lab/")


Mounted at /content/gdrive


In [2]:
!pip install lime

# Object detector

In [3]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, fasterrcnn_resnet50_fpn
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from lime.lime_image import LimeImageExplainer
from src.faster_rcnn import fasterrcnn_resnet18_fpn

from src.utils import jaccard

# device = 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# our dataset has two classes only - background and person
num_classes = 2

In [4]:
# load an instance segmentation model pre-trained pre-trained on COCO
# model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True, num_classes=num_classes)
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, num_classes=num_classes)
model = fasterrcnn_resnet18_fpn(pretrained_backbone=True, num_classes=num_classes)
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# load model checkpoint
path = os.path.join(os.getcwd(), "./checkpoints/faster_rcnn_resnet1830_epochs.ckpt")
checkpoint = torch.load(path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

model.to(device) 

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride

# PennFudan Dataset

In [5]:
import sys
sys.path.append("./detection")
from pennfudan_dataset import PennFudanDataset, get_transform

# use our dataset and defined transformations
dataset = PennFudanDataset('./PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('./PennFudanPed', get_transform(train=False))
# changing to array


# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

In [6]:
#creating test image



# COCO Dataset

# Visualize the Detection



In [ ]:
# pick one image from the test set
img, _ = dataset_test[0]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
from PIL import Image, ImageDraw

result = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
image = ImageDraw.Draw(result)  

for bbox in prediction[0]["boxes"]:
    image.rectangle(bbox.cpu().numpy().tolist(), outline ="red")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.imshow(result)

NameError: ignored

# Explanations

## Import Explainer

In [7]:
from src.sodexplainer import SODExplainer
explainer_sodex = SODExplainer(model=model)

## Lime Explanation

In [ ]:
image_test = dataset_test[0][0]
image_test = image_test.permute(1,2,0).detach().numpy()
image_test = image_test.astype('double')

In [ ]:
explanation = explainer_sodex.get_lime_explanation(image_test,dataset_test[0], num_samples=100000)

In [ ]:
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries

temp_1, mask_1 = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=20, hide_rest=True)
temp_2, mask_2 = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=200, hide_rest=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax1.axis('off')
ax2.axis('off')

## Rise explanation

In [7]:
image_test = dataset_test[0][0].permute(1,2,0)
image_test = image_test.detach().numpy().astype('double')
sal, preds, masks = explainer_sodex.get_rise_explanation(torch.from_numpy(image_test), N=10, s=10, p1=0.5)

Explaining:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Explaining: 100%|██████████| 10/10 [00:52<00:00,  5.23s/it]


In [ ]:
import matplotlib.pyplot as plt
plt.axis('off')
plt.imshow(sal, cmap='jet', alpha=0.8)
plt.show()

In [ ]:
sal.shape

(364, 579)

# Evaluation

In [8]:
from src.eval import *

## Pointing Game Accuracy

Run Rise explanation for 5 images with N=5 (just for testing).

In [9]:
num_images = 5
sals = []
for i in range(num_images):
  image_test = dataset_test[i][0].permute(1,2,0)
  image_test = image_test.detach().numpy().astype('double')
  sal, preds, masks = explainer_sodex.get_rise_explanation(torch.from_numpy(image_test), N=5, s=10, p1=0.5) # <- change N later
  sals.append(sal)

Explaining:   0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Explaining: 100%|██████████| 5/5 [00:21<00:00,  4.33s/it]


Compute pointing game accuracy.

In [10]:
from src.eval import compute_localization_acc
dataset_test = PennFudanDataset('./PennFudanPed', get_transform(train=False))
dataset_test = torch.utils.data.Subset(dataset_test, indices[:num_images])
evaluate(sals,dataset_test)

0.2